In [1]:
import re
import json
import requests
import datetime
import pandas as pd
from googletrans import Translator
from sqlalchemy import create_engine
# from collections import defaultdict

In [2]:
url = 'https://3g.dxy.cn/newh5/view/pneumonia'

* ### Get the China Province and City Data (just for preview)

In [3]:
# use regular expression to get the html infomation
raw_html = requests.get(url).content.decode('utf8')
match = re.search('window.getAreaStat = (.*?)}catch', raw_html)
raw_json = match.group(1)
result1 = json.loads(raw_json, encoding='utf8')

## Part 1: Get the World Wide Total Data

In [4]:
# use regular expression to get the html infomation
raw_html = requests.get(url).content.decode('utf8')
match = re.search('window.getStatisticsService = (.*?)}catch', raw_html)
raw_json = match.group(1)
result2 = json.loads(raw_json, encoding='utf8')

In [5]:
result2

{'id': 1,
 'createTime': 1579537899000,
 'modifyTime': 1582167974000,
 'infectSource': '该字段已替换为说明2',
 'passWay': '该字段已替换为说明3',
 'imgUrl': 'https://img1.dxycdn.com/2020/0201/450/3394153392393266839-135.png',
 'dailyPic': 'https://img1.dxycdn.com/2020/0211/763/3395998884005602079-135.png,https://img1.dxycdn.com/2020/0211/362/3395998896890788910-135.png,https://img1.dxycdn.com/2020/0211/365/3395998905480724211-135.png,https://img1.dxycdn.com/2020/0211/364/3395998916217859778-135.png,https://img1.dxycdn.com/2020/0211/922/3395998929103046444-135.png,https://img1.dxycdn.com/2020/0211/089/3395998939840182072-135.png',
 'dailyPics': ['https://img1.dxycdn.com/2020/0211/763/3395998884005602079-135.png',
  'https://img1.dxycdn.com/2020/0211/362/3395998896890788910-135.png',
  'https://img1.dxycdn.com/2020/0211/365/3395998905480724211-135.png',
  'https://img1.dxycdn.com/2020/0211/364/3395998916217859778-135.png',
  'https://img1.dxycdn.com/2020/0211/922/3395998929103046444-135.png',
  'https://im

#### __Table 1: Summary Dataframe__

In [6]:
df_China_total = pd.DataFrame.from_dict([result2])
df_China_total = df_China_total [["confirmedCount","suspectedCount","deadCount","curedCount"]]
df_China_total["Country"] ="China"
df_China_total["Continent"] = "Asia"
new_order=["Country","Continent","confirmedCount","suspectedCount","deadCount","curedCount"]
df_China_total=df_China_total[new_order]
# df_summary = df_summary[["confirmedCount","suspectedCount","deadCount","curedCount"]]
# df_summary=df_summary.T.reset_index()
# df_summary=df_summary.rename(columns={"index": "Count", 0: "Total"})
df_China_total

,Country,Continent,confirmedCount,suspectedCount,deadCount,curedCount
0,China,Asia,74675,4922,2121,16169


#### __Table 1: Summary Dataframe(Database)__

In [ ]:
#example:
#group_export.to_sql(con=engine, name=example_table, if_exists='replace', 
#                     flavor='mysql', index=False)

# with engine.connect() as con:
#     con.execute('ALTER TABLE `example_table` ADD PRIMARY KEY (`ID_column`);')

In [7]:
# export df to sqlite
engine = create_engine('sqlite:///wuhan_pneumonia.sqlite')
df_China_total.to_sql('China_total', engine, index=False, if_exists='replace')

In [8]:
# Check if table in sqlite
df_chinatotal = pd.read_sql_query('SELECT * FROM China_total',engine)
df_chinatotal

,Country,Continent,confirmedCount,suspectedCount,deadCount,curedCount
0,China,Asia,74675,4922,2121,16169


## Part 2: Get the Province Data

In [9]:
# use regular expression to get the html infomation
raw_html = requests.get(url).content.decode('utf8')
# get the json info within html code
match = re.search('getListByCountryTypeService1 = (.*?)}catch', raw_html)
raw_json = match.group(1)
result3 = json.loads(raw_json, encoding='utf8')

In [10]:
result3[0]

{'id': 330277,
 'createTime': 1582167974000,
 'modifyTime': 1582167974000,
 'tags': '',
 'countryType': 1,
 'continents': '',
 'provinceId': '54',
 'provinceName': '西藏自治区',
 'provinceShortName': '西藏',
 'cityName': '',
 'currentConfirmedCount': 0,
 'confirmedCount': 1,
 'suspectedCount': 0,
 'curedCount': 1,
 'deadCount': 0,
 'comment': '',
 'sort': 0,
 'operator': 'liyilin',
 'locationId': 540000}

In [11]:
df_provinces_original = pd.DataFrame.from_dict(result3)
df_provinces_original.info()
print(df_provinces_original.columns)
df_provinces_original.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 19 columns):
id                       34 non-null int64
createTime               34 non-null int64
modifyTime               34 non-null int64
tags                     34 non-null object
countryType              34 non-null int64
continents               34 non-null object
provinceId               34 non-null object
provinceName             34 non-null object
provinceShortName        34 non-null object
cityName                 34 non-null object
currentConfirmedCount    34 non-null int64
confirmedCount           34 non-null int64
suspectedCount           34 non-null int64
curedCount               34 non-null int64
deadCount                34 non-null int64
comment                  34 non-null object
sort                     34 non-null int64
operator                 34 non-null object
locationId               34 non-null int64
dtypes: int64(11), object(8)
memory usage: 5.2+ KB
Index(['id', 'createT

,id,createTime,modifyTime,tags,countryType,continents,provinceId,provinceName,provinceShortName,cityName,currentConfirmedCount,confirmedCount,suspectedCount,curedCount,deadCount,comment,sort,operator,locationId
0,330277,1582167974000,1582167974000,,1,,54,西藏自治区,西藏,,0,1,0,1,0,,0,liyilin,540000
1,330262,1582167974000,1582167974000,,1,,63,青海省,青海,,2,18,0,16,0,,40,liyilin,630000
2,330253,1582167974000,1582167974000,确诊 2 例,1,,67,澳门,澳门,,4,10,0,6,0,,23,liyilin,820000
3,330248,1582167974000,1582167974000,确诊 1 例,1,,68,台湾,台湾,,21,24,0,2,1,,27,liyilin,710000
4,330260,1582167974000,1582167974000,确诊 2 例,1,,62,甘肃省,甘肃,,24,91,0,65,2,,25,liyilin,620000


In [12]:
# remove the last 2 rows of the dataframe
df_provinces_r = df_provinces_original.drop(df_provinces_original.head(0).index)
df_provinces_t = df_provinces_r[['provinceId','provinceShortName','confirmedCount', 'suspectedCount', 'curedCount', 'deadCount','locationId']]
df_provinces_t.head()

,provinceId,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,locationId
0,54,西藏,1,0,1,0,540000
1,63,青海,18,0,16,0,630000
2,67,澳门,10,0,6,0,820000
3,68,台湾,24,0,2,1,710000
4,62,甘肃,91,0,65,2,620000


In [13]:
translator = Translator()
df_provinces_t["Provinces"] = df_provinces_t["provinceShortName"].map(lambda x: translator.translate(x, src="zh-CN", dest="en").text)
# Add the new translation column to the data frame and check if there is any error
df_provinces_t

C:\Users\Mason\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,provinceId,provinceShortName,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Provinces
0,54,西藏,1,0,1,0,540000,Tibet
1,63,青海,18,0,16,0,630000,Qinghai
2,67,澳门,10,0,6,0,820000,Macao
3,68,台湾,24,0,2,1,710000,Taiwan
4,62,甘肃,91,0,65,2,620000,Gansu Province
5,64,宁夏,71,0,42,0,640000,Ningxia
6,22,吉林,91,0,40,1,220000,Jilin
7,65,新疆,76,0,20,1,650000,Xinjiang
8,66,香港,65,0,5,2,810000,Hong Kong
9,21,辽宁,121,0,58,1,210000,Liaoning Province


In [14]:
# Since 陕西 and 山西 have the same translation, change 陕西 as Shaanxi
#df.iloc[0, df.columns.get_loc('col2')] = 100
df_provinces_t.iloc[16,df_provinces_t.columns.get_loc('Provinces') ] = 'Shaanxi'
# drop the Chinese Provinces names column
df_provinces_english=df_provinces_t.drop(columns='provinceShortName')
df_provinces_english

C:\Users\Mason\Anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,provinceId,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Provinces
0,54,1,0,1,0,540000,Tibet
1,63,18,0,16,0,630000,Qinghai
2,67,10,0,6,0,820000,Macao
3,68,24,0,2,1,710000,Taiwan
4,62,91,0,65,2,620000,Gansu Province
5,64,71,0,42,0,640000,Ningxia
6,22,91,0,40,1,220000,Jilin
7,65,76,0,20,1,650000,Xinjiang
8,66,65,0,5,2,810000,Hong Kong
9,21,121,0,58,1,210000,Liaoning Province


In [15]:
# remove the extra province in province name if the translation is not accurate
new = [i.replace(" Province","") for i in df_provinces_english["Provinces"]]

df_provinces_m=df_provinces_english.replace(list(df_provinces_t["Provinces"]),new)
df_provinces_m

,provinceId,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Provinces
0,54,1,0,1,0,540000,Tibet
1,63,18,0,16,0,630000,Qinghai
2,67,10,0,6,0,820000,Macao
3,68,24,0,2,1,710000,Taiwan
4,62,91,0,65,2,620000,Gansu
5,64,71,0,42,0,640000,Ningxia
6,22,91,0,40,1,220000,Jilin
7,65,76,0,20,1,650000,Xinjiang
8,66,65,0,5,2,810000,Hong Kong
9,21,121,0,58,1,210000,Liaoning


In [16]:
# Read the China provinces coordinates .csv file
df_China_coordinates = pd.read_csv("data/China.csv")
df_China_coordinates.Provinces

0           Shanghai
1            Beijing
2          Guangdong
3              Hubei
4            Tianjin
5          Chongqing
6           Liaoning
7            Sichuan
8            Shaanxi
9            Guangxi
10           Jiangsu
11           Guizhou
12      Heilongjiang
13            Fujian
14             Jilin
15          Shandong
16          Zhejiang
17            Yunnan
18            Shanxi
19             Henan
20             Hunan
21             Gansu
22             Hebei
23           Jiangxi
24          Xinjiang
25            Hainan
26    Inner Mongolia
27             Anhui
28           Qinghai
29           Ningxia
30             Tibet
Name: Provinces, dtype: object

In [17]:
# Since the coordinates does not have Macao, Taiwan, HongKong we add these to present great China area
listOfSeries = [pd.Series(['Macao', 'China', 22.1987, 113.5439], index=df_China_coordinates.columns ) ,
                pd.Series(['Taiwan', 'China', 23.6978, 120.9605], index=df_China_coordinates.columns) ,
                pd.Series(['Hong Kong', 'China', 22.3193, 114.1694], index=df_China_coordinates.columns) ]

In [18]:
df_great_China = df_China_coordinates.append(listOfSeries , ignore_index=True)
df_great_China.tail()

,Provinces,country,lat,lng
29,Ningxia,China,38.468056,106.273056
30,Tibet,China,29.650000,91.100000
31,Macao,China,22.198700,113.543900
32,Taiwan,China,23.697800,120.960500
33,Hong Kong,China,22.319300,114.169400


#### __Table 2: Great China Data__

In [19]:
# Merge the coordinates with dataframe for future use
df_China_provinces = pd.merge(df_provinces_m, df_great_China, how='outer', on = ["Provinces"])
df_China_provinces.columns
new_order = ['provinceId', 'Provinces','confirmedCount', 'suspectedCount', 'curedCount','deadCount', 'country', 'lat', 'lng', 'locationId']
df_China_provinces=df_China_provinces[new_order]
df_China_provinces=df_China_provinces.sort_values(by='confirmedCount', ascending=False)
df_China_provinces.head()

,provinceId,Provinces,confirmedCount,suspectedCount,curedCount,deadCount,country,lat,lng,locationId
33,42,Hubei,62031,0,10337,2029,China,30.583333,114.266667,420000
32,44,Guangdong,1332,0,619,5,China,23.116667,113.250000,440000
31,41,Henan,1265,0,614,19,China,34.683611,113.532500,410000
30,33,Zhejiang,1175,0,609,0,China,30.293650,120.161419,330000
27,43,Hunan,1010,0,578,4,China,28.200000,112.966667,430000


#### __Table 2: Great China Data Table (Database)__

In [20]:
df_China_provinces.to_sql('Great_China', engine, index=False,if_exists='replace')

In [21]:
# Check if table in sqlite
df_Great_China_db = pd.read_sql_query('SELECT * FROM Great_China',engine)
df_Great_China_db.head()

,provinceId,Provinces,confirmedCount,suspectedCount,curedCount,deadCount,country,lat,lng,locationId
0,42,Hubei,62031,0,10337,2029,China,30.583333,114.266667,420000
1,44,Guangdong,1332,0,619,5,China,23.116667,113.250000,440000
2,41,Henan,1265,0,614,19,China,34.683611,113.532500,410000
3,33,Zhejiang,1175,0,609,0,China,30.293650,120.161419,330000
4,43,Hunan,1010,0,578,4,China,28.200000,112.966667,430000


## Part 3: Get the World Data

In [22]:
# use regular expression to get the html infomation
raw_html = requests.get(url).content.decode('utf8')
# get the json info within html code
match = re.search('getListByCountryTypeService2 = (.*?)}catch', raw_html)
raw_json = match.group(1)
result4 = json.loads(raw_json, encoding='utf8')

In [23]:
result4[0]

{'id': 330306,
 'createTime': 1582167974000,
 'modifyTime': 1582167974000,
 'tags': '',
 'countryType': 2,
 'continents': '其他',
 'provinceId': '10',
 'provinceName': '钻石公主号邮轮',
 'provinceShortName': '',
 'cityName': '',
 'currentConfirmedCount': 619,
 'confirmedCount': 621,
 'suspectedCount': 0,
 'curedCount': 0,
 'deadCount': 2,
 'comment': '',
 'sort': 0,
 'operator': 'liyilin',
 'locationId': 0}

In [24]:
df_world_original = pd.DataFrame.from_dict(result4)
df_world_original.info()
print(df_world_original.columns)
df_world_original.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 19 columns):
id                       27 non-null int64
createTime               27 non-null int64
modifyTime               27 non-null int64
tags                     27 non-null object
countryType              27 non-null int64
continents               27 non-null object
provinceId               27 non-null object
provinceName             27 non-null object
provinceShortName        27 non-null object
cityName                 27 non-null object
currentConfirmedCount    27 non-null int64
confirmedCount           27 non-null int64
suspectedCount           27 non-null int64
curedCount               27 non-null int64
deadCount                27 non-null int64
comment                  27 non-null object
sort                     27 non-null int64
operator                 27 non-null object
locationId               27 non-null int64
dtypes: int64(11), object(8)
memory usage: 4.1+ KB
Index(['id', 'createT

,id,createTime,modifyTime,tags,countryType,continents,provinceId,provinceName,provinceShortName,cityName,currentConfirmedCount,confirmedCount,suspectedCount,curedCount,deadCount,comment,sort,operator,locationId
0,330306,1582167974000,1582167974000,,2,其他,10,钻石公主号邮轮,,,619,621,0,0,2,,0,liyilin,0
1,330267,1582167974000,1582167974000,,2,亚洲,6,日本,,,71,84,0,12,1,,0,liyilin,951002
2,330268,1582167974000,1582167974000,,2,亚洲,7,韩国,,,66,82,0,16,0,,0,liyilin,951004
3,330264,1582167974000,1582167974000,,2,亚洲,3,新加坡,,,50,84,0,34,0,,0,liyilin,952009
4,330263,1582167974000,1582167974000,,2,亚洲,2,泰国,,,20,35,0,15,0,,0,liyilin,952010


In [29]:
df_world_t = df_world_original[['id','provinceName','continents',  'confirmedCount','suspectedCount', 'curedCount', 'deadCount','locationId']]
df_world_t.head()
df_world_t = df_world_t.drop([0])
df_world_t.head()

,id,provinceName,continents,confirmedCount,suspectedCount,curedCount,deadCount,locationId
1,330267,日本,亚洲,84,0,12,1,951002
2,330268,韩国,亚洲,82,0,16,0,951004
3,330264,新加坡,亚洲,84,0,34,0,952009
4,330263,泰国,亚洲,35,0,15,0,952010
5,330275,德国,欧洲,16,0,1,0,963003


In [30]:
translator = Translator()
df_world_t["Country"] = df_world_t["provinceName"].map(lambda x: translator.translate(x, src="zh-CN", dest="en").text)
# Add the new translation column to the data frame and check if there is any error
df_world_t["Continent"] = df_world_t["continents"].map(lambda x: translator.translate(x, src="zh-CN", dest="en").text)
df_world_t

,id,provinceName,continents,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Country,Continent
1,330267,日本,亚洲,84,0,12,1,951002,Japan,Asia
2,330268,韩国,亚洲,82,0,16,0,951004,Korea,Asia
3,330264,新加坡,亚洲,84,0,34,0,952009,Singapore,Asia
4,330263,泰国,亚洲,35,0,15,0,952010,Thailand,Asia
5,330275,德国,欧洲,16,0,1,0,963003,Germany,Europe
6,330269,美国,北美洲,15,0,3,0,971002,United States,North America
7,330270,越南,亚洲,16,0,7,0,952011,Vietnam,Asia
8,330265,马来西亚,亚洲,22,0,15,0,952007,Malaysia,Asia
9,330266,法国,欧洲,12,0,4,1,961002,France,Europe
10,330273,加拿大,北美洲,8,0,2,0,971001,Canada,North America


In [31]:
df_world_english=df_world_t.drop(columns=['provinceName','continents'])
df_world_english.iloc[3,df_world_english.columns.get_loc('Country') ] = 'South Korea'
df_world_english

,id,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Country,Continent
1,330267,84,0,12,1,951002,Japan,Asia
2,330268,82,0,16,0,951004,Korea,Asia
3,330264,84,0,34,0,952009,Singapore,Asia
4,330263,35,0,15,0,952010,South Korea,Asia
5,330275,16,0,1,0,963003,Germany,Europe
6,330269,15,0,3,0,971002,United States,North America
7,330270,16,0,7,0,952011,Vietnam,Asia
8,330265,22,0,15,0,952007,Malaysia,Asia
9,330266,12,0,4,1,961002,France,Europe
10,330273,8,0,2,0,971001,Canada,North America


In [32]:
df_world_coordinates = pd.read_csv("data/world.csv")
df_world_coordinates

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [33]:
df_world_coordinates=df_world_coordinates.rename(columns={"country": "Countryshort", "name": "Country"})
df_world_coordinates.head()

,Countryshort,latitude,longitude,Country
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


In [34]:
df_world_c = pd.merge(df_world_english, df_world_coordinates, how='inner', on = ["Country"])
df_world_c

,id,confirmedCount,suspectedCount,curedCount,deadCount,locationId,Country,Continent,Countryshort,latitude,longitude
0,330267,84,0,12,1,951002,Japan,Asia,JP,36.204824,138.252924
1,330264,84,0,34,0,952009,Singapore,Asia,SG,1.352083,103.819836
2,330263,35,0,15,0,952010,South Korea,Asia,KR,35.907757,127.766922
3,330275,16,0,1,0,963003,Germany,Europe,DE,51.165691,10.451526
4,330269,15,0,3,0,971002,United States,North America,US,37.090240,-95.712891
5,330270,16,0,7,0,952011,Vietnam,Asia,VN,14.058324,108.277199
6,330265,22,0,15,0,952007,Malaysia,Asia,MY,4.210484,101.975766
7,330266,12,0,4,1,961002,France,Europe,FR,46.227638,2.213749
8,330273,8,0,2,0,971001,Canada,North America,CA,56.130366,-106.346771
9,330272,15,0,10,0,990001,Australia,Oceania,AU,-25.274398,133.775136


#### __Table 3: World Data__

In [35]:
df_world_c.columns
new_order2 = ['id', 'Country', 'Continent', 'Countryshort','confirmedCount','suspectedCount', 'curedCount', 'deadCount','latitude', 'longitude','locationId']
df_world=df_world_c[new_order2]
df_world

,id,Country,Continent,Countryshort,confirmedCount,suspectedCount,curedCount,deadCount,latitude,longitude,locationId
0,330267,Japan,Asia,JP,84,0,12,1,36.204824,138.252924,951002
1,330264,Singapore,Asia,SG,84,0,34,0,1.352083,103.819836,952009
2,330263,South Korea,Asia,KR,35,0,15,0,35.907757,127.766922,952010
3,330275,Germany,Europe,DE,16,0,1,0,51.165691,10.451526,963003
4,330269,United States,North America,US,15,0,3,0,37.090240,-95.712891,971002
5,330270,Vietnam,Asia,VN,16,0,7,0,14.058324,108.277199,952011
6,330265,Malaysia,Asia,MY,22,0,15,0,4.210484,101.975766,952007
7,330266,France,Europe,FR,12,0,4,1,46.227638,2.213749,961002
8,330273,Canada,North America,CA,8,0,2,0,56.130366,-106.346771,971001
9,330272,Australia,Oceania,AU,15,0,10,0,-25.274398,133.775136,990001


#### __Table 3: World Data Table (Database)__

In [36]:
df_world.to_sql('world', engine, index=False, if_exists='replace')

In [37]:
# Check if table in sqlite
df_worlddb = pd.read_sql_query('SELECT * FROM world',engine)
df_worlddb.head()

,id,Country,Continent,Countryshort,confirmedCount,suspectedCount,curedCount,deadCount,latitude,longitude,locationId
0,330267,Japan,Asia,JP,84,0,12,1,36.204824,138.252924,951002
1,330264,Singapore,Asia,SG,84,0,34,0,1.352083,103.819836,952009
2,330263,South Korea,Asia,KR,35,0,15,0,35.907757,127.766922,952010
3,330275,Germany,Europe,DE,16,0,1,0,51.165691,10.451526,963003
4,330269,United States,North America,US,15,0,3,0,37.090240,-95.712891,971002


#### __Table 4: World Summary Table__

In [38]:
df_China_total

,Country,Continent,confirmedCount,suspectedCount,deadCount,curedCount
0,China,Asia,74675,4922,2121,16169


In [39]:
df_world_total = df_world[["Country","Countryshort","Continent","confirmedCount","suspectedCount","curedCount","deadCount"]]
df_world_total.head()

,Country,Countryshort,Continent,confirmedCount,suspectedCount,curedCount,deadCount
0,Japan,JP,Asia,84,0,12,1
1,Singapore,SG,Asia,84,0,34,0
2,South Korea,KR,Asia,35,0,15,0
3,Germany,DE,Europe,16,0,1,0
4,United States,US,North America,15,0,3,0


In [40]:
all_dfs = [df_China_total,df_world_total]
world_total_df=pd.concat(all_dfs).reset_index(drop=True)
world_total_df.head()
world_total_df=world_total_df.fillna("CN")

C:\Users\Mason\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


#### __Table 4: World Summary Table (Database)__

In [41]:
world_total_df.to_sql('world_China', engine, index=False,if_exists='replace')

In [42]:
df_worldcdb = pd.read_sql_query('SELECT * FROM world_China',engine)
df_worldcdb.head()

,Continent,Country,Countryshort,confirmedCount,curedCount,deadCount,suspectedCount
0,Asia,China,CN,74675,16169,2121,4922
1,Asia,Japan,JP,84,12,1,0
2,Asia,Singapore,SG,84,34,0,0
3,Asia,South Korea,KR,35,15,0,0
4,Europe,Germany,DE,16,1,0,0


#### __Table 5: Summary Table (Database)__

In [43]:
world_total_df.sum()

Continent         AsiaAsiaAsiaAsiaEuropeNorth AmericaAsiaAsiaEur...
Country           ChinaJapanSingaporeSouth KoreaGermanyUnited St...
Countryshort      CNJPSGKRDEUSVNMYFRCAAUAEITGBSEPHEGNPKHLKFIINRU...
confirmedCount                                                75022
curedCount                                                    16297
deadCount                                                      2126
suspectedCount                                                 4922
dtype: object

In [44]:
world_total=pd.DataFrame(world_total_df.sum()).T

In [45]:
world_total=world_total.drop(['Continent','Countryshort'], axis=1)
world_total

,Country,confirmedCount,curedCount,deadCount,suspectedCount
0,ChinaJapanSingaporeSouth KoreaGermanyUnited St...,75022,16297,2126,4922


In [46]:
world_total.iloc[0,world_total.columns.get_loc('Country') ] = 'world'
world_total

,Country,confirmedCount,curedCount,deadCount,suspectedCount
0,world,75022,16297,2126,4922


In [47]:
# export df to sqlite
engine = create_engine('sqlite:///wuhan_pneumonia.sqlite')
world_total.to_sql('summary', engine, index=False, if_exists='replace')

In [48]:
df_summarydb = pd.read_sql_query('SELECT * FROM summary',engine)
df_summarydb

,Country,confirmedCount,curedCount,deadCount,suspectedCount
0,world,75022,16297,2126,4922
